# Parte 1: carregando a base de dados e criando o problema.

Primeiro, vamos importar três funções: <br>
- read_documents, responsável por puxar os textos da base de dados. <br>
- print_search_result, responsável por
- run_search, responsável por executar alguma busca especificada pelo usuário. <br>

Depois, importaremos duas classes: <br>
- LanguageModel, responsável por administrar a criação do modelo de linguagem com seus n-gramas.
- CompleteSentence, que herda de LanguageModel e faz a interface com o problema de busca.

In [ ]:
from utils import read_documents, print_search_result
from Searches import run_search
from UpperClasses import LanguageModel
from completeSentence import CompleteSentence
from completeSentence2 import CompleteSentence2

   As opções que temos de documentos são, em ordem crescente de tamanho:
- "safatle"
- "flor"
- "mmorpho"
- "saka"
- "machado"
- "todos", a agregação de todos os anteriores.


A seguir, precisamos definir um N para o nosso modelo. Note que o modelo também cria os (N-1)-grama para podermos, no futuro, calcular a probabilidade condicional de um dado N-grama.

In [ ]:
N = 3
print(f"Creating a {N}-gram Language Model based on safatle document")
text1 = read_documents("safatle")
model1 = LanguageModel(text1, N)

In [ ]:
print(f"Creating a {N}-gram Language Model based on machado document")
text2 = read_documents("machado")
model2 = LanguageModel(text2, N)

In [ ]:
print(f"Creating a {N}-gram Language Model based on all documents")
text3 = read_documents("all")
model3 = LanguageModel(text3, N)

Com esses modelos, nós podemos acessar a contagem de cada N e (N-1)-gramas.

In [ ]:
most_common_count = model3.n_grams.most_common(1)[0]
print(f"The most common n-gram together with its count was: {most_common_count}")
most_common = most_common_count[0]
most_common_smaller = model3.n_grams_smaller.most_common()[0][0]
print(f"In the first model, the n-gram {most_common} occurs {model1.n_grams[most_common]}, and the (n-1)-gram {most_common_smaller} occurs {model1.n_grams_smaller[most_common_smaller]}")
print(f"In the second model, the n-gram {most_common} occurs {model2.n_grams[most_common]}, and the (n-1)-gram {most_common_smaller} occurs {model2.n_grams_smaller[most_common_smaller]}")
print(f"In the third model, the n-gram {most_common} occurs {model3.n_grams[most_common]}, and the (n-1)-gram {most_common_smaller} occurs {model3.n_grams_smaller[most_common_smaller]}")

# Parte 2: modelando como busca.

Nós iremos agora definir completar uma frase como um problema de busca. <br>
A nossa classe CompleteSentence herda de Language Model, por isso sua inicialização será semelhante ao que vimos anteriormente. <br>
Para sua inicialização precisaremos de três parâmetros: um documento (string), um N (int), e uma sentença (string). 

In [ ]:
cs = CompleteSentence(text3, N, "<s> <s> <s>")

A classe CompleteSentence também herda da classe Problem vista em aula. <br>
Existem 5 funções herdadas que devem ser implementadas. <br>
A primeira é a initialState(), que deve retornar um estado baseado na frase inicial passada no momento da instanciação do objeto.

In [ ]:
print(f"The initial state of the problem is: {cs.initialState()}")

A segunda função é a actions(state), a qual deve retornar todas as ações que podem ser efetuadas sobre state.

In [ ]:
initial = cs.initialState()
options = cs.actions(initial)
print(f"The initial state has a branching factor of {len(options)}.")

A terceira e a quarta função são a result(state, action) e cost(state1, action, state2). <br>
A função result é responsável por receber um estado1 e uma ação e retornar um estado2, resultante de aplicar a ação sobre estado1. <br>
A função cost(state1, action, state2) retorna o custo de ir do estado state1 para estado state2 através de action. <br>

In [ ]:
for index, op in enumerate(options):
    v = cs.result(initial, op)
    print(f"{initial} ---'{op}'---> {v} costs {cs.cost(initial, op, v):.2f}")
    if index == 5:
        print("There may be more actions, but we are stopping here...")
        break
        

A última função é a responsável por identificação dos estados meta.
Certifique-se que a sua função isGoal(state) retorna verdadeiro se state for meta e false caso contrário.

In [ ]:
fake_goal = cs.initialState()
true_goal = cs.result(cs.result(fake_goal, "."), "</s>")
assert cs.isGoal(fake_goal) == False, "Initial state is not a goal"
assert cs.isGoal(true_goal) == True, f"The state {true_goal} is a goal"

# Parte 3: Executando as buscas

Para executarmos as buscas, nós iremos utilizar a função run_search. Esta função recebe o nome de uma busca, uma instância de problema e possivelmente mais parâmetros. <br>


Primeiro, executaremos uma busca de custo uniforme. Nessa busca, cada nó tem como prioridade a probabilidade total que leva até ele. <br>
Com essa busca nós encontramos a resposta ótima. Note que ela é relativamente pequena, pois quanto maior a sentença, menos provável ela é. <br>

In [ ]:
S = run_search("UniformCost", cs)
print_search_result(S)

Nós podemos tentar gerar frases mais longas fazendo uma busca em profundidade limitada. <br>
As possíveis vantagens de utilizar essa busca, além de gerar uma frase mais longa, é utilizar menos memória. <br>
A desvantagem é que não garantimos mais que encontraremos a resposta ótima. <br>

In [ ]:
for i in range(4, 7):
    print("Maximum depth: ", i)
    S2 = run_search("LimitedDepthSearch", cs, i)
    print_search_result(S2)

Um método comum para geração de sentenças é o Beam Search. <br>
Esse algoritmo é um tipo de busca local e segue o pseudo-código à seguir: <br>

Esse método é mais rápido que os anteriores e gera candidatas diferentes de resposta. <br>
Suas desvantagens são que ele não possui garantias de encontrar a resposta ótima e nem que ele vai encontrar alguma resposta. <br>
Como as respostas geradas são de tamanho diferente, a métrica utilizada para compará-las não é a probabilidade, mas sim a perplexidade. <br>
Perplexidade de uma sentença é definida como o -log da probabilidade dessa sentença dividida pelo tamanho da sentença. <br>

In [ ]:
S3 = run_search("BeamSearch", cs, 15)

O Beam Search retorna uma lista de triplas (histórico, probabilidade, perplexidade), onde histórico contém todos os estados que foram utilizados até chegar naquele estado meta. <br>
Como a definição de estado varia de pessoa para pessoa, fica a cargo de vocês completar a função para que a sentença final seja apresentada corretamente. <br>

In [ ]:
def print_beam_search(solutions):
    """
    Change this function so that it prints the proper sentence from h
    """
    for solution in solutions:
        h, probability, perplexity = solution
        sentence = ""
        for state in h:
            for word in state:
                sentence += word+" "
        print(f"The sentence: {sentence} has probability {probability:.2f}, size {len(h)}, thus {perplexity:.2f} of perplexity")

In [ ]:
print_beam_search(S3)

Note que agora a função deixou de ser monotônica: ao expandirmos um nó, os filhos dele podem ter custo menor que do filho.

# Parte 4: Mudando o problema

Implemente a classe CompleteSentencePerplexity para que agora possamos executar um problema de busca utilizando a perplexidade ao invés da probabilidade. <br>

O resultado das próximas duas células deve ser exatamente igual ao do anterior.

In [ ]:
cs2 = CompleteSentence2(text3, N, "<s> <s> <s>")

In [ ]:
cs2.initialState()

In [ ]:
initial = cs2.initialState()
options = cs2.actions(initial)
print(f"The initial state has a branching factor of {len(options)}.")

O resultado da próxima célula deve ser como o da vez anterior, mas com todos os custos divididos pela mesma constante.

In [ ]:
for index, op in enumerate(options):
    v = cs2.result(initial, op)
    print(f"{initial} ---'{op}'---> {v} costs {cs2.cost(initial, op, v):.2f}")
    if index == 5:
        print("There may be more actions, but we are stopping here...")
        break

A próxima celula deve continuar dando os mesmos resultados do anterior.

In [ ]:
fake_goal = cs2.initialState()
true_goal = cs2.result(cs2.result(fake_goal, "."), "</s>")
assert cs2.isGoal(fake_goal) == False, "Initial state is not a goal"
assert cs2.isGoal(true_goal) == True, f"The state {true_goal} is a goal"

# Parte 5: reexecutando as buscas

Com essa nossa nova função de custo, a busca por custo uniforme não mais garante resposta ótima, já que uma alternativa ruim pode gerar descendentes melhores.

In [ ]:
S4 = run_search("UniformCost", cs2)
print_search_result(S4)

A Busca em Profundidade Iterativa continua com todas as mesmas propriedades, já que todos os elementos são comparados pelo nível que estão na árvore de busca.

In [ ]:
for i in range(4, 7):
    print("Maximum depth: ", i)
    S5 = run_search("LimitedDepthSearch", cs2, i)
    print_search_result(S5)

O Beam Search também continua encontrando as mesmas respostas que anteriormente, pois em toda iteração todas as candidatas possuem mesmo tamanho.

In [ ]:
S = run_search("BeamSearch", cs2, 15)

In [ ]:
def print_beam_search2(solutions):
    """
    Change this function so that it prints the proper sentence from h
    """
    for solution in solutions:
        h, probability, perplexity = solucao
        sentence = ""
        for state in h:
            for word in state:
                sentence += word+" "
        print(f"The sentence: {sentence} has probability {probability:.2f}, size {len(h)}, thus {perplexity:.2f} of perplexity")

In [ ]:
print_beam_search2(S)